<a href="https://colab.research.google.com/github/Inha-AI/DACON-semiconductor-competition/blob/feature%2FYoonSungLee/submission_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp36-none-any.whl size=10032 sha256=85647bcf12d9ab5a77163421fa0229c7d01d1a03f19d1ececa26092efaff7544
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# 데이터 자료형을 적절히 변형시켜 데이터의 크기를 줄이는 방법

# for col in df_train.columns:
#     col_type = df_train[col].dtypes
#     min1 = df_train[col].min()
#     max1 = df_train[col].max()
#     if str(col_type)[:3] == 'int':
#         df_train[col] = df_train[col].astype(np.int16)
#     else:
#         if min1 > np.finfo(np.float16).min and max1 < np.finfo(np.float16).max:
#             df_train[col] = trdf_trainain[col].astype(np.float16)
#         elif min1 > np.finfo(np.float32).min and max1 < np.finfo(np.float32).max:
#             df_train[col] = df_train[col].astype(np.float32)
#         else:
#             df_train[col] = df_train[col].astype(np.float64)

In [0]:
df_train = pd.read_csv('/gdrive/My Drive/DACON-semiconductor-competition/dataset/train.csv')
df_test = pd.read_csv('/gdrive/My Drive/DACON-semiconductor-competition/dataset/test.csv')

In [0]:
# 독립변수와 종속변수를 분리합니다.

train_X = df_train.iloc[:,4:]
train_Y = df_train.iloc[:,0:4]
test_X = df_test.iloc[:,1:]

In [0]:
# train set을 shuffle하여 다시 train set과 validation set으로 분리합니다.

# train_X, val_X, train_Y, val_Y = train_test_split(train_X, train_Y, test_size=0.25)

# Model 16

* 지금까지 좋은 성능을 보였던 모델들을 모두 불러와 (각 모델들의 예측값들의 합)/(모델의 수) 를 계산하여 '앙상블 기법'을 적용한 결과값 도출

In [0]:
def swish(x) :
    return x * keras.activations.sigmoid(x)

In [0]:
# 좋은 성능을 보였던 모델을 모두 불러옵니다.

from keras.models import load_model

model_12_200 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_12.h5', custom_objects={'swish':swish})
model_12_300 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_12_300.h5', custom_objects={'swish':swish})
model_13_200 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_13.h5', custom_objects={'swish':swish})
model_13_250 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_13_250.h5', custom_objects={'swish':swish})
model_13_300 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_13_300.h5', custom_objects={'swish':swish})
model_14_200 = load_model('/gdrive/My Drive/DACON-semiconductor-competition/model_14.h5', custom_objects={'swish':swish})

In [0]:
# 좋은 성능을 보였던 모델들의 성능을 평가합니다.
# 모델들간의 큰 성능의 차이가 없으므로 6개의 모델을 모두 앙상블 방법으로 적용합니다.

model_list = [model_12_200, model_12_300, model_13_200, model_13_250, model_13_300, model_14_200]

model_evaluate = {}
for model in model_list:
    results = model.evaluate(train_X, train_Y, batch_size=630)
    model_evaluate[model] = results
    print('test loss, test acc: ', results)

810000/810000 [==============================] - 9s 12us/step
test loss, test acc:  [1.2413994020753436, 0.967248135275311]
810000/810000 [==============================] - 8s 9us/step
test loss, test acc:  [1.0118190819687314, 0.971292579167419]
810000/810000 [==============================] - 8s 10us/step
test loss, test acc:  [1.1997492327226533, 0.9680110984974437]
810000/810000 [==============================] - 8s 10us/step
test loss, test acc:  [0.9881053525739246, 0.9650604800979297]
810000/810000 [==============================] - 8s 10us/step
test loss, test acc:  [1.2487017318142786, 0.9687123325169087]
810000/810000 [==============================] - 9s 11us/step
test loss, test acc:  [1.0525636264218226, 0.9689110983444584]


In [0]:
# 예측값을 생성합니다.

model_predict = []
for model in model_list:
    model_predict.append(model.predict(test_X))

pred_test = sum(model_predict)/6

In [0]:
# submission 파일을 생성합니다.
sample_sub = pd.read_csv('/gdrive/My Drive/DACON-semiconductor-competition/dataset/sample_submission.csv', index_col=0)
submission = sample_sub+pred_test
submission.to_csv('/gdrive/My Drive/DACON-semiconductor-competition/submission_16.csv')

### Bayesian Optimization
http://research.sualab.com/introduction/practice/2019/02/19/bayesian-optimization-overview-1.html<br>
http://research.sualab.com/introduction/practice/2019/04/01/bayesian-optimization-overview-2.html<br>

### Swish Activation
https://www.machinecurve.com/index.php/2019/05/30/why-swish-could-perform-better-than-relu/#todays-activation-functions